In [78]:
import pandas as pd
import requests
import os
import re

from bs4 import BeautifulSoup
from datetime import datetime
#from pdfminer

# Link and File Paths

In [79]:
ODCY_LINK = "https://childcaresearch.ohio.gov/search?q=fVLNbhMxEN40v0uapgKhHhAiBy6VQkUR1xwWN1VDIVl1V0gFcXDWk42FY6%2b83pS98Q6ICxdeg1fgyBvwJjB2uhCJqrPS2DOf59tvPPZqnuf9RrOrtd0ddCRhyRXMiVqtlBwO3oDOuZKj46On9hsOSCFMoWEkoTCaiuEgLOaCJ%2bdQxuoDyJEshGhaxkfbREcn8ewoAqqTJdHcgOb0MZ7ZC7VacwZ6WqzmoFtEFdKUDcJN2X7LM6IY7FZH4jKD1gU1XKbdkGrDqZjSFbQnckGlyXuXWJvGijGBknszYUuuIz%2fUkCdLpYQfuSVIoX9Cy3y2mGWgkVPJ%2fTNV6O1E8zVQke%2b%2fgIXSsCkjVEN3vAaJGuy%2bE13x1QqD%2b1EGiRUEwHKy5IJZeC%2fWVOaZ0sYR9oMFNv6PqTdbg5Y8XRob3TnlIFiseZYfjKkWpaPBs2z8MUMaJPDPgLLIYO%2f3Qs3X1MA5l9hmihmQB5tBiPK0wCT7q%2bLuK07nXOCVTmReoKIEmtNgfEma0zEJA9wTEqI%2fJaROZkEjINEEB5FKThsThLxaDce5eSPthnejNTudmrOH%2fw3dXesFlSm8e2%2ffV%2b1Gs8jOk%2bNW3bL59cpVL9MFm51V4DdvloHWt3h7q7TtztrIIo0Htwi0IvyWdZbA71jn2%2b5s7rZCp9cNJub4IseS2bUTgYDEAHM8rkUv%2b%2frymV1%2fPf%2fCnC77p9Z25vvn4JtDOhVynfl0%2bPPwh0P8CqmsYu3%2bAQ%3d%3d"

REL_PATH = "https://childcaresearch.ohio.gov/"

# Helper Functions

In [80]:
def extract_html(url):
    """
    Parse the html at the given url into a beautiful soup object, for manipulation.
    
    :param url: Any valid URL
    :return: the parsed HTML or None if the request failed
    """ 
    
    response = requests.get(url)
    
    # check if the request was successful (status code 200)
    if response.status_code == 200:

        # parse the HTML content 
        soup = BeautifulSoup(response.text, 'html.parser')
        
        return soup
    else:
        print(f"Failed to retrieve page, status code: {response.status_code}")
        return None

In [81]:
def extract_inspection(url):
    """
    Extract the inspection link from the childcare center page
    
    :param url: the child care center page
    :return: the inspection page link
    """
    inspection_url = None

    # get the html for the program page
    program_page = extract_html(url)

    if program_page:
        inspection_button_span = program_page.find('span', class_='inspectionsButton')

        if inspection_button_span:
            inspection_link_tag = inspection_button_span.find_parent('a')

            if inspection_link_tag and 'href' in inspection_link_tag.attrs:
                inspection_url = REL_PATH + inspection_link_tag['href']
                
    return inspection_url

In [82]:
def extract_pdf(url) -> str:
    """
    Extract the pdf link from the inspection page
    :param url: inspection url
    :return: the pdf link
    """

    inspection_page = extract_html(url)
    most_recent_pdf_link = None
    most_recent_date = None
    
    if inspection_page is not None:
        rows = inspection_page.find_all('div', class_='resultsListRow')
        
        for row in rows:
            date_column = row.find('div', class_='resultsListColumn')

            pdf_col = row.find('span', class_='inspectionPDFlink') 
            pdf_link_tag = pdf_col.find('a', href=True)
            
            if pdf_link_tag and date_column:
                
                # format into a datetime object for date comparisons
                inspection_date = list(date_column)[2].strip()
                inspection_date = datetime.strptime(inspection_date, "%m/%d/%Y")
                pdf_link = pdf_link_tag['href']
                
                # only save the most recent date (may not be necessary, since all appear to be listed in order. Adds robustness though...)
                if most_recent_date is None or inspection_date > most_recent_date:
                    most_recent_date = inspection_date
                    most_recent_pdf_link = REL_PATH + pdf_link
    
    return most_recent_pdf_link

In [109]:
def extract_all_pdfs(url):

    pdf_urls = []
    main_page = None
    page_num = 1
    
    # loop for all available pages
    while not (pdf_urls and main_page is None):
        
        # get the current page of results
        main_page = extract_html(f"{url}&{page_num}")
        
        if main_page is not None:
            # get all results rows for further processing
            results_list = main_page.find('div', class_='resultsList') 
            rows = results_list.find_all('div', class_='resultsListRow')
            
            for row in rows:
                program_name_column = row.find('div', class_='resultsListColumn programListColumnName')

                program_df = pd.DataFrame()
                if program_name_column:
                    
                    program_link_tag = program_name_column.find('a')
                    
                    if program_link_tag:
                        program_name = program_link_tag.text.strip()
                        program_url = REL_PATH + program_link_tag['href']
                        inspection_url = extract_inspection(program_url)  
                        program_pdf_link = extract_pdf(inspection_url) if inspection_url is not None else None
                        program_df['program_name'] = [program_name]
                        program_df['pdf'] = [program_pdf_link]
                        
                address_columns = row.findAll("div", class_="resultsListColumn")
                if address_columns:
                    program_df['Address'] = [address_columns[1].get_text(strip=True)]
                    program_df['City'] = [address_columns[2].get_text(strip=True)]
                    program_df['Zip'] = [address_columns[3].get_text(strip=True)]
                    
                # save the current row  information
                pdf_urls.append(program_df)

            break
            
        # next page 
        page_num += 1
        
    # combine into a single dataframe
    url_df = pd.concat(pdf_urls, axis=0) 
    
    # return with the program name as the index
    return url_df.set_index("program_name")


In [ ]:
def download_pdf(pdf_url):
    """
    Create a temporary pdf file for data extraction 
    :param pdf_url: pdf to download
    :return: 
    """

    response = requests.get(pdf_url)
    if response.status_code == 200:
        return BytesIO(response.content)
    else:
        print(f"Failed to download PDF: {response.status_code}")
        return None

# Extract PDF Links

In [105]:
test_link = "https://childcaresearch.ohio.gov/search?q=fVLNbhMxEN40v0uapgKhHhAiBy6VQkUR1xwWN1VDIVl1V0gFcXDWk42FY6%2b83pS98Q6ICxdeg1fgyBvwJjB2uhCJqrPS2DOf59tvPPZqnuf9RrOrtd0ddCRhyRXMiVqtlBwO3oDOuZKj46On9hsOSCFMoWEkoTCaiuEgLOaCJ%2bdQxuoDyJEshGhaxkfbREcn8ewoAqqTJdHcgOb0MZ7ZC7VacwZ6WqzmoFtEFdKUDcJN2X7LM6IY7FZH4jKD1gU1XKbdkGrDqZjSFbQnckGlyXuXWJvGijGBknszYUuuIz%2fUkCdLpYQfuSVIoX9Cy3y2mGWgkVPJ%2fTNV6O1E8zVQke%2b%2fgIXSsCkjVEN3vAaJGuy%2bE13x1QqD%2b1EGiRUEwHKy5IJZeC%2fWVOaZ0sYR9oMFNv6PqTdbg5Y8XRob3TnlIFiseZYfjKkWpaPBs2z8MUMaJPDPgLLIYO%2f3Qs3X1MA5l9hmihmQB5tBiPK0wCT7q%2bLuK07nXOCVTmReoKIEmtNgfEma0zEJA9wTEqI%2fJaROZkEjINEEB5FKThsThLxaDce5eSPthnejNTudmrOH%2fw3dXesFlSm8e2%2ffV%2b1Gs8jOk%2bNW3bL59cpVL9MFm51V4DdvloHWt3h7q7TtztrIIo0Htwi0IvyWdZbA71jn2%2b5s7rZCp9cNJub4IseS2bUTgYDEAHM8rkUv%2b%2frymV1%2fPf%2fCnC77p9Z25vvn4JtDOhVynfl0%2bPPwh0P8CqmsYu3%2bAQ%3d%3d&p=1"

In [110]:
pdf_links = extract_all_pdfs(test_link)
pdf_links

,pdf,Address,City,Zip
program_name,,,,
A BRIGHT START 4 KIDZ LEARNING CTR,https://childcaresearch.ohio.gov//pdf/00224002...,8211 PLATT,CLEVELAND,44104
A BRIGHTER START CHILDCARE,https://childcaresearch.ohio.gov//pdf/00000020...,2765 BLUE ROCK RD.,CINCINNATI,45239
A CHILD'S GARDEN,https://childcaresearch.ohio.gov//pdf/00000020...,5427 JULMAR DRIVE,CINCINNATI,45238
A CHILD'S JOURNEY LEARNING CENTER,https://childcaresearch.ohio.gov//pdf/00217001...,846 S. YEARLING RD,WHITEHALL,43213
A CHILD'S PLACE LEARNING CENTER,https://childcaresearch.ohio.gov//pdf/00000040...,2010 OFFICEVIEW PLACE,REYNOLDSBURG,43068
A GREAT START PRESCHOOL INC,https://childcaresearch.ohio.gov//pdf/00000020...,7001 FAR HILLS AVE,DAYTON,45459
A JOYFUL JOURNEY ACADEMY,https://childcaresearch.ohio.gov//pdf/00222002...,1536 BARNETT ROAD,COLUMBUS,43227
A JUBILEE ACADEMY,https://childcaresearch.ohio.gov//pdf/00000030...,15751 LAKESHORE BLVD,CLEVELAND,44110
A KIDS ONLY EARLY LEARNING CENTER INC. 4,https://childcaresearch.ohio.gov//pdf/00219001...,2505 SOUTH RIDGE EAST,ASHTABULA,44004


In [97]:
test_df

,pdf
program_name,
A BRIGHT START 4 KIDZ LEARNING CTR,https://childcaresearch.ohio.gov//pdf/00224002...
A BRIGHTER START CHILDCARE,https://childcaresearch.ohio.gov//pdf/00000020...
A CHILD'S GARDEN,https://childcaresearch.ohio.gov//pdf/00000020...
A CHILD'S JOURNEY LEARNING CENTER,https://childcaresearch.ohio.gov//pdf/00217001...
A CHILD'S PLACE LEARNING CENTER,https://childcaresearch.ohio.gov//pdf/00000040...
A GREAT START PRESCHOOL INC,https://childcaresearch.ohio.gov//pdf/00000020...
A JOYFUL JOURNEY ACADEMY,https://childcaresearch.ohio.gov//pdf/00222002...
A JUBILEE ACADEMY,https://childcaresearch.ohio.gov//pdf/00000030...
A KIDS ONLY EARLY LEARNING CENTER INC. 4,https://childcaresearch.ohio.gov//pdf/00219001...
